### Author: Akshay Ijantkar
### Team: Aqua Wizards
### Project: Surfers Bible

In [ ]:
https://api.meteostat.net/v1/stations/nearby?lat=1.283&lon=103.83&limit=5&key=XXXXXXXX
    
{
    "meta": {
        "source": "National Oceanic and Atmospheric Administration, Deutscher Wetterdienst"
    },
    "data": [
        {
            "id": "95748",
            "name": "Wollongong Airport / Albion Park",
            "distance": "5.3"
        }
    ]
}



In [ ]:
# !pip install --user pygeocoder
# !pip install --user geopandas
# !pip install --user geopy
# !pip install  googlemaps
# !pip install weather-au
# !pip install pyowm

# Import Libraries:

In [32]:
import pandas as pd
import numpy as np
import random
import matplotlib as plt
from matplotlib import pyplot
import seaborn as sns; sns.set()
from scipy.stats import norm 
import matplotlib.pyplot as plt
# For Linear regression
from sklearn.linear_model import LinearRegression
# For split given dataset into train and test set.
from sklearn.model_selection import train_test_split
# To verify models using this metrics 
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
# v
from matplotlib import rcParams
rcParams['figure.figsize'] = 50,50
import pandas_profiling
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas import ExcelWriter
from pandas import ExcelFile

from pygeocoder import Geocoder

import sys
from weather_au import api
from weather_au import summary
from weather import place, observations, uv_index
import time

import re

%matplotlib inline

# Load CSV clean addresses with Coordinates CSV as Dataframe:

In [2]:
dataset_path = r"D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\\"
raw_shark_df = pd.read_csv(dataset_path + "shark_file_geolocation_checkpoint.csv")
# raw_shark_df = pd.read_excel(dataset_path + "shark_file_geolocation_checkpoint.xlsx")
print("raw_shark_df.shape = ", raw_shark_df.shape)

raw_shark_df.head(5)

raw_shark_df.shape =  (1318, 31)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Gender,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Address,lat,lng,location_type,postal_code,formatted_address,place_id
0,2020.01.17,17-Jan-20,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""","B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.17-Schroeter.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.17,2020.01.17,6502.0,NaN,NaN,Windang Beach New South Wales AUSTRALIA,-34.534325,150.874566,APPROXIMATE,NaN,NaN,NaN
1,2020.01.07,7-Jan-20,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,Miah Holmes,F,7,Lacerations to leg,N,17h30,Lemon shark,"B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.07-Holmes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.07,2020.01.07,6498.0,NaN,NaN,North West Island Queensland AUSTRALIA,-23.295088,151.707872,APPROXIMATE,4805,"North West Island, Queensland 4805, Australia",ChIJE1RBGQxA6GsR0Vxvqw9Ilvk
2,2020.01.05,5-Jan-20,2020,Unprovoked,AUSTRALIA,Western Australia,Cull Island / Esperance,Scuba diving,Gary Johnson,M,57,FATAL,Y,13h00,White shark,"B. Myatt, GSAF",2020.01.05-Johnson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.05,2020.01.05,6497.0,NaN,NaN,Cull Island / Esperance Western Australia AUST...,-33.922179,121.903218,APPROXIMATE,NaN,NaN,NaN
3,2020.01.02,2-Jan-20,2020,Questionable,AUSTRALIA,Western Australia,Esperance,Swimming after being swept off rocks,Eric Birighitti,M,21,Probable drowning and scavenging,Y,16h10,NaN,"B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.02-Birighitti.pdf,harkattackfile.net/spreadsheets/pdf_directory/...,harkattackfile.net/spreadsheets/pdf_directory/...,2020.01.02,2020.01.02,6495.0,NaN,NaN,Esperance Western Australia AUSTRALIA,-33.861310,121.891424,APPROXIMATE,6450,"Esperance WA 6450, Australia",ChIJuycfmldHWyoR0At6JDj2AAU
4,2019.12.30,30-Dec-19,2019,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,male,M,30s,Minor injury to hand,N,12h58,"Shovelnose ""shark"" which is a ray, not a shark)","B. Myatt, GSAF",2019.12.30-Queensland.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2019.12.30,2019.12.30,6494.0,NaN,NaN,North West Island Queensland AUSTRALIA,-23.295088,151.707872,APPROXIMATE,4805,"North West Island, Queensland 4805, Australia",ChIJE1RBGQxA6GsR0Vxvqw9Ilvk


### Sincer we dont need that level of precision so rounding coordinates upto 3 decimals:

In [3]:
raw_shark_df['lat'] = round(raw_shark_df['lat'], 3)
raw_shark_df['lng'] = round(raw_shark_df['lng'], 3)
raw_shark_df['lat_lon']= raw_shark_df[['lat', 'lng']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
raw_shark_df.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Gender,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Address,lat,lng,location_type,postal_code,formatted_address,place_id,lat_lon
1313,1807.01.12,12-Jan-1807,1807,Unprovoked,AUSTRALIA,New South Wales,"Cockle Bay, Sydney Harbour",NaN,male,NaN,NaN,Survived,N,NaN,NaN,"J. Green, p.31",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cockle Bay, Sydney Harbour New South Wales AUS...",-33.866,151.201,APPROXIMATE,NaN,NaN,NaN,-33.866_151.201
1314,1804.02.26.R,Reported 26-Feb-1804,1804,Watercraft,AUSTRALIA,New South Wales,"Georges Head, off Port Jackson",NaN,boat,NaN,NaN,No injury to occupants,N,NaN,NaN,"Sydney Gazette, 2/26/1804",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Georges Head, off Port Jackson New South Wales...",-33.846,151.249,APPROXIMATE,NaN,NaN,NaN,-33.846_151.249
1315,1803.03.00,Mar-1803,1803,Unprovoked,AUSTRALIA,Western Australia,"Hamelin Harbour, at Faure Island",NaN,M. Lefevre & a sailor (rescuer),NaN,NaN,Shark knocked him down & tore clothing of the ...,N,NaN,NaN,F. Peron ref in G.P. Whitley (Fishes of Austra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hamelin Harbour, at Faure Island Western Austr...",-25.844,113.894,APPROXIMATE,Australia,"Faure Island, Shark Bay WA 6537, Australia",ChIJf4C1l2Yn4SsRMMWKBzn2AA8,-25.844_113.894
1316,1791.00.00,1791,1791,Unprovoked,AUSTRALIA,New South Wales,Port Jackson,NaN,"female, an Australian aboriginal",NaN,NaN,"FATAL, ""bitten in two""",Y,NaN,NaN,"G.P. Whitley; D. Baldridge, p.162",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Port Jackson New South Wales AUSTRALIA,-33.846,151.249,APPROXIMATE,NaN,NaN,NaN,-33.846_151.249
1317,1788.05.10,10-May-1788,1788,Watercraft,AUSTRALIA,New South Wales,Sydney Harbor,Fishing,boat,NaN,NaN,"No injury to occupants, shark bit oar and rudder",N,NaN,NaN,"G.P. Whitley citing J. Cobley, Sydney Cove, p....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sydney Harbor New South Wales AUSTRALIA,-33.844,151.237,APPROXIMATE,NaN,NaN,NaN,-33.844_151.237


# Copy Clean Case Number as new Date:

In [4]:
raw_shark_df['Case_Date'] = raw_shark_df['Case Number']
raw_shark_df.head(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Gender,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Address,lat,lng,location_type,postal_code,formatted_address,place_id,lat_lon,Case_Date
0,2020.01.17,17-Jan-20,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""","B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.17-Schroeter.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.17,2020.01.17,6502.0,NaN,NaN,Windang Beach New South Wales AUSTRALIA,-34.534,150.875,APPROXIMATE,NaN,NaN,NaN,-34.534_150.875,2020.01.17
1,2020.01.07,7-Jan-20,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,Miah Holmes,F,7,Lacerations to leg,N,17h30,Lemon shark,"B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.07-Holmes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.07,2020.01.07,6498.0,NaN,NaN,North West Island Queensland AUSTRALIA,-23.295,151.708,APPROXIMATE,4805,"North West Island, Queensland 4805, Australia",ChIJE1RBGQxA6GsR0Vxvqw9Ilvk,-23.295_151.708,2020.01.07
2,2020.01.05,5-Jan-20,2020,Unprovoked,AUSTRALIA,Western Australia,Cull Island / Esperance,Scuba diving,Gary Johnson,M,57,FATAL,Y,13h00,White shark,"B. Myatt, GSAF",2020.01.05-Johnson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.05,2020.01.05,6497.0,NaN,NaN,Cull Island / Esperance Western Australia AUST...,-33.922,121.903,APPROXIMATE,NaN,NaN,NaN,-33.922_121.903,2020.01.05


# Clean Case_Date:

### Remove tailing letter from case dates:

In [5]:
re.sub("(.[a-zA-Z])|(-[a-zA-Z])", '',    '2019.04.27-a') 

'2019.04.27'

In [6]:
def remove_tailing_letters_func(Case_Date):
    return re.sub("(.[a-zA-Z])|(-[a-zA-Z])", '',Case_Date)

raw_shark_df['Case_Date'] = raw_shark_df['Case_Date'].apply(remove_tailing_letters_func)
raw_shark_df.tail(25)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Gender,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Address,lat,lng,location_type,postal_code,formatted_address,place_id,lat_lon,Case_Date
1293,1852.10.09.R,Reported 09-Oct-1852,1852,Unprovoked,AUSTRALIA,New South Wales,Coalcliff,Fell overboard,James Rogers,NaN,NaN,FATAL,Y,NaN,NaN,"Maitland Mercury, 10/09/1852",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coalcliff New South Wales AUSTRALIA,-34.246,150.973,APPROXIMATE,2508,"Coalcliff NSW 2508, Australia",ChIJz15iVwzeEmsRAOIyFmh9AQU,-34.246_150.973,1852.10.09
1294,1849.12.09,09-Dec-1849,1849,Unprovoked,AUSTRALIA,New South Wales,Sydney Harbor,NaN,male,NaN,NaN,FATAL,Y,NaN,NaN,"T.H. Huxley's Diary, 1935, pp.285-286; G.P. Wh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sydney Harbor New South Wales AUSTRALIA,-33.844,151.237,APPROXIMATE,NaN,NaN,NaN,-33.844_151.237,1849.12.09
1295,1849.12.01,01-Dec-1849,1849,Unprovoked,AUSTRALIA,New South Wales,"Woolloomooloo, Sydney Harbor (Estuary)",Bathing,a sailor from the steamship Eagle,NaN,NaN,FATAL,Y,Afternoon,NaN,"Moreton Bay Courier, 12/15/1849",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Woolloomooloo, Sydney Harbor (Estuary) New Sou...",-33.852,151.211,GEOMETRIC_CENTER,Australia,"Sydney Harbour Bridge, Sydney Harbour Bridge, ...",ChIJ49XqJV2uEmsRPsTAF7eOlGg,-33.852_151.211,1849.12.01
1296,1849.09.19,19-Sep-1849,1849,Unprovoked,AUSTRALIA,Victoria,Warrnambool,Swimming,a sailor from the schooner Brother,NaN,NaN,FATAL,Y,NaN,NaN,Maitland Mercury & Hunter River General Advert...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Warrnambool Victoria AUSTRALIA,-38.369,142.498,APPROXIMATE,NaN,NaN,NaN,-38.369_142.498,1849.09.19
1297,1849.01.27,27-Jan-1849,1849,Invalid,AUSTRALIA,New South Wales,Sydney,boat capsized,William Henry Elliott,NaN,NaN,Torso bitten but may have been postmortem,NaN,NaN,Shark involvement prior to death unconfirmed,"Maitland Mercury, 2/3/1849",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sydney New South Wales AUSTRALIA,-33.869,151.209,APPROXIMATE,NaN,NaN,NaN,-33.869_151.209,1849.01.27
1298,1847.11.30,30-Nov-1847,1847,Unprovoked,AUSTRALIA,Queensland,Brisbane River,Swimming,James Stewart,NaN,NaN,Thigh & calf bitten,N,NaN,NaN,"Moreton Bay Courier, 12.4/1847",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brisbane River Queensland AUSTRALIA,-26.895,152.319,APPROXIMATE,NaN,NaN,NaN,-26.895_152.319,1847.11.30
1299,1847.03.11,11-Mar-1847,1847,Sea Disaster,AUSTRALIA,Queensland,Moreton Bay,Wreck of the Sovereign,Spicer,NaN,NaN,Foot severed,N,NaN,NaN,"The Queenslander, 3/12/1921",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moreton Bay Queensland AUSTRALIA,-27.312,153.345,APPROXIMATE,NaN,NaN,NaN,-27.312_153.345,1847.03.11
1300,1847.00.00.c,Reported in 1847,1847,Unprovoked,AUSTRALIA,Queensland,Moreton Bay,NaN,a native,NaN,NaN,Foot severed at ankle joint,N,NaN,NaN,Narrative of the Voyage of HMS Rattlesnake 184...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moreton Bay Queensland AUSTRALIA,-27.312,153.345,APPROXIMATE,NaN,NaN,NaN,-27.312_153.345,1847.00.00
1301,1845.12.26,26-Dec-1845,1845,Unprovoked,AUSTRALIA,New South Wales,Newcastle,Fishing,John Williams,NaN,NaN,FATAL,Y,NaN,NaN,The Maitland Mercury & Hunter River General Ad...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Newcastle New South Wales AUSTRALIA,-32.928,151.782,APPROXIMATE,NaN,NaN,NaN,-32.928_151.782,1845.12.26
1302,1842.00.00.a,1842,1842,Unprovoked,AUSTRALIA,New South Wales,Sydney Harbor,"Unknown, but it was said to be the ""First know...",male,NaN,NaN,Recovered,N,NaN,NaN,"V.M. Coppleson (1933), p.450",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sydney Harbor New South Wales AUSTRALIA,-33.844,151.237,APPROXIMATE,NaN,NaN,NaN,-33.844_151.237,1842.00.00


### Remove rows where month and day == "00":

In [7]:
raw_shark_df.isnull().sum()

Case Number                  0
Date                         0
Year                         0
Type                         1
Country                      0
Area                         0
Location                     0
Activity                   111
Name                        18
Gender                    1310
Age                        578
Injury                       5
Fatal (Y/N)                109
Time                       695
Species                    523
Investigator or Source       2
pdf                        830
href formula               831
href                       831
Case Number.1              831
Case Number.2              831
original order             831
Unnamed: 22               1318
Unnamed: 23               1318
Address                      0
lat                          0
lng                          0
location_type                0
postal_code                467
formatted_address          467
place_id                   467
lat_lon                      0
Case_Dat

In [8]:
%%time
for index, row in raw_shark_df.iterrows():
    month = row["Case_Date"].split(".")[1]
    day = row["Case_Date"].split(".")[2]
    
    if(month == "00") | (day == "00"):
        raw_shark_df.at[index, "Case_Date"] = None        
        print(row["Case_Date"])

2018.05.00
2004.00.00
1996.03.00
1995.00.00
1993.04.00
1991.11.00
1991.01.00
1988.00.00
1981.00.00
1978.04.00
1974.00.00
1971.00.00
1969.02.00
1969.01.00
1966.04.00
1962.00.00
1961.04.00
1961.00.00
1959.01.00
1956.03.00
1955.08.00
1955.03.00
1955.02.00
1953.12.00
1953.10.00
1950.00.00
1949.12.00
1949.03.00
1947.11.00
1946.04.00
1942.01.00
1942.00.00
1936.01.00
1936.00.00
1935.06.00
1934.03.00
1932.05.00
1930.12.00
1929.00.00
1929.00.00
1929.00.00
1927.00.00
1927.00.00
1918.00.00
1913.01.00
1913.00.00
1909.00.00
1895.00.00
1892.00.00
1890.00.00
1888.12.00
1886.03.00
1886.00.00
1882.00.00
1878.00.00
1876.00.00
1876.00.00
1875.00.00
1871.00.00
1871.00.00
1871.00.00
1870.00.00
1868.00.00
1865.00.00
1864.00.00
1863.04.00
1855.00.00
1853.00.00
1847.00.00
1842.00.00
1840.12.00
1840.02.00
1839.00.00
1825.00.00
1825.00.00
1803.03.00
1791.00.00
Wall time: 123 ms


In [9]:
raw_shark_df.isnull().sum()

Case Number                  0
Date                         0
Year                         0
Type                         1
Country                      0
Area                         0
Location                     0
Activity                   111
Name                        18
Gender                    1310
Age                        578
Injury                       5
Fatal (Y/N)                109
Time                       695
Species                    523
Investigator or Source       2
pdf                        830
href formula               831
href                       831
Case Number.1              831
Case Number.2              831
original order             831
Unnamed: 22               1318
Unnamed: 23               1318
Address                      0
lat                          0
lng                          0
location_type                0
postal_code                467
formatted_address          467
place_id                   467
lat_lon                      0
Case_Dat

In [10]:
raw_shark_df.dropna(subset = ['Case_Date'], 
                    how='all', 
                    inplace=True)
raw_shark_df.reset_index(drop = True, 
                                              inplace = True)
raw_shark_df.shape

(1241, 33)

In [11]:
raw_shark_df.isnull().sum()

Case Number                  0
Date                         0
Year                         0
Type                         1
Country                      0
Area                         0
Location                     0
Activity                    93
Name                        18
Gender                    1233
Age                        516
Injury                       4
Fatal (Y/N)                102
Time                       623
Species                    479
Investigator or Source       2
pdf                        769
href formula               770
href                       770
Case Number.1              770
Case Number.2              770
original order             770
Unnamed: 22               1241
Unnamed: 23               1241
Address                      0
lat                          0
lng                          0
location_type                0
postal_code                432
formatted_address          432
place_id                   432
lat_lon                      0
Case_Dat

In [12]:
raw_shark_df.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Gender,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,Address,lat,lng,location_type,postal_code,formatted_address,place_id,lat_lon,Case_Date
1236,1832.01.23.R,Reported 23-Jan-1832,1832,Unprovoked,AUSTRALIA,New South Wales,Sydney,Bathing,male,NaN,NaN,Laceration to leg,N,NaN,NaN,"Sydney Herald, 1/23/1832",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sydney New South Wales AUSTRALIA,-33.869,151.209,APPROXIMATE,NaN,NaN,NaN,-33.869_151.209,1832.01.23
1237,1831.01.22.R,Reported 22- Jan-1831,1831,Invalid,AUSTRALIA,Tasmania,Hobart,"Boat capsized, clinging to line",Robert Dudlow,NaN,NaN,"Drowned, no shark involvement",NaN,NaN,Invalid,"C. Black, GSAF; Sydney Gazette, 1/22/1831",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hobart Tasmania AUSTRALIA,-42.882,147.327,APPROXIMATE,NaN,NaN,NaN,-42.882_147.327,1831.01.22
1238,1807.01.12,12-Jan-1807,1807,Unprovoked,AUSTRALIA,New South Wales,"Cockle Bay, Sydney Harbour",NaN,male,NaN,NaN,Survived,N,NaN,NaN,"J. Green, p.31",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cockle Bay, Sydney Harbour New South Wales AUS...",-33.866,151.201,APPROXIMATE,NaN,NaN,NaN,-33.866_151.201,1807.01.12
1239,1804.02.26.R,Reported 26-Feb-1804,1804,Watercraft,AUSTRALIA,New South Wales,"Georges Head, off Port Jackson",NaN,boat,NaN,NaN,No injury to occupants,N,NaN,NaN,"Sydney Gazette, 2/26/1804",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Georges Head, off Port Jackson New South Wales...",-33.846,151.249,APPROXIMATE,NaN,NaN,NaN,-33.846_151.249,1804.02.26
1240,1788.05.10,10-May-1788,1788,Watercraft,AUSTRALIA,New South Wales,Sydney Harbor,Fishing,boat,NaN,NaN,"No injury to occupants, shark bit oar and rudder",N,NaN,NaN,"G.P. Whitley citing J. Cobley, Sydney Cove, p....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sydney Harbor New South Wales AUSTRALIA,-33.844,151.237,APPROXIMATE,NaN,NaN,NaN,-33.844_151.237,1788.05.10


In [14]:
raw_shark_df[['lat','lng', 'Case_Date']]

,lat,lng,Case_Date
0,-34.534,150.875,2020.01.17
1,-23.295,151.708,2020.01.07
2,-33.922,121.903,2020.01.05
3,-33.861,121.891,2020.01.02
4,-23.295,151.708,2019.12.30
5,-34.583,150.867,2019.12.20
6,-21.931,114.124,2019.12.06
7,-24.831,152.459,2019.11.30
8,-24.917,152.483,2019.11.23
9,-20.268,148.717,2019.10.29


# Aeris API and Maps
* Access ID: 0UaN5DgaG7sw9S5bAiEqR

* Secret Key: yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ

In [50]:
# raw_shark_df
import json
import requests
# response = requests.get("https://api.aerisapi.com/observations/archive/-34.53,150.87?&from=17/01/2020&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ")
response = requests.get("https://api.aerisapi.com/observations/archive/-33.86,121.89?&from=14/09/2000&format=json&filter=allstations&limit=1&client_id=<>&client_secret=<>")
weat_dict = json.loads(response.text)
weat_dict

{'success': True,
 'error': {'code': 'warn_invalid_param',
  'description': 'Invalid from parameter has been adjusted.'},
 'response': {'id': 'AUS_AUS109504',
  'dataSource': 'AUSBOM',
  'loc': {'long': 121.8971, 'lat': -33.8707},
  'place': {'name': 'esperance ntc aws',
   'city': 'esperance harbour',
   'state': 'wa',
   'country': 'au'},
  'periods': [{'ob': {'type': 'station',
     'timestamp': 1586448600,
     'dateTimeISO': '2020-04-10T00:10:00+08:00',
     'recTimestamp': 1586449235,
     'recDateTimeISO': '2020-04-10T00:20:35+08:00',
     'tempC': 21.7,
     'tempF': 71,
     'dewpointC': None,
     'dewpointF': None,
     'humidity': None,
     'pressureMB': None,
     'pressureIN': None,
     'spressureMB': None,
     'spressureIN': None,
     'altimeterMB': None,
     'altimeterIN': None,
     'windKTS': None,
     'windKPH': None,
     'windMPH': None,
     'windSpeedKTS': None,
     'windSpeedKPH': None,
     'windSpeedMPH': None,
     'windDirDEG': None,
     'windDir': N

In [51]:
%%time
ACCESS_ID = ""
SECRET_KEY = ""

weather_df = pd.DataFrame(columns = ['date', 'lat', 'lng'])
for col in list(weat_dict['response']['periods'][0]['ob'].keys()):
    weather_df[col] = ""
    
cnt = 0
for index, row in raw_shark_df[['lat','lng', 'Case_Date']].iterrows():
    if index <= 500:
        
        day = row["Case_Date"].split(".")[2]
        month = row["Case_Date"].split(".")[1]
        year = row["Case_Date"].split(".")[0]
        get_request = ""
        get_request += "https://api.aerisapi.com/observations/archive/"
        get_request += str(row["lat"])
        get_request += ","
        get_request += str(row["lng"])
        get_request += "?&from="
        get_request += day
        get_request += "/"
        get_request += month
        get_request += "/"
        get_request += year
        get_request += "&format=json&filter=allstations&limit=1&client_id="
        get_request += ACCESS_ID
        get_request += "&client_secret="
        get_request += SECRET_KEY
        
        response_dict = json.loads(requests.get(get_request).text)
        print("get_request = ", get_request)
        print("****************************************************************************************************")
#         print("response_dict = ",response_dict)
        
        if len(response_dict['response']) != 0:
            cnt = cnt + 1
            print("cnt =>>>>>>>>>>>>>>>>>>>>> ", cnt)
            weather_row_dict = {}
            
            weather_row_dict['date'] = row["Case_Date"]
            weather_row_dict['lat'] = row["lat"]
            weather_row_dict['lng'] = row["lng"]
            
            for weather_ob in response_dict['response']['periods']:
#                 print(weather_ob['ob'])
                for weather_attri in weather_ob['ob'].keys():
                    weather_row_dict[weather_attri] = weather_ob['ob'][weather_attri]
                    
#                     print(weather_attri, " = ", weather_ob['ob'][weather_attri])  
                    
                weather_df = weather_df.append(weather_row_dict, ignore_index=True)
                    
        print("\n")
        time.sleep(1)
# weather_df            

get_request =  https://api.aerisapi.com/observations/archive/-34.534,150.875?&from=17/01/2020&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************


get_request =  https://api.aerisapi.com/observations/archive/-23.295,151.708?&from=07/01/2020&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************


get_request =  https://api.aerisapi.com/observations/archive/-33.922,121.903?&from=05/01/2020&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************


get_request =  https://api.aeri



get_request =  https://api.aerisapi.com/observations/archive/-30.666,152.732?&from=09/12/2018&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  12


get_request =  https://api.aerisapi.com/observations/archive/-12.182,136.782?&from=18/11/2018&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  13


get_request =  https://api.aerisapi.com/observations/archive/-34.77,150.813?&from=17/11/2018&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
***********************************************************************



get_request =  https://api.aerisapi.com/observations/archive/-38.356,145.248?&from=17/02/2018&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  34


get_request =  https://api.aerisapi.com/observations/archive/-34.682,150.851?&from=15/02/2018&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  35


get_request =  https://api.aerisapi.com/observations/archive/-26.681,153.122?&from=14/02/2018&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
**********************************************************************



get_request =  https://api.aerisapi.com/observations/archive/-34.072,150.718?&from=19/03/2017&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  55


get_request =  https://api.aerisapi.com/observations/archive/-31.954,115.862?&from=25/02/2017&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  56


get_request =  https://api.aerisapi.com/observations/archive/-32.579,115.654?&from=24/02/2017&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
**********************************************************************



get_request =  https://api.aerisapi.com/observations/archive/-16.744,145.671?&from=04/07/2016&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  79


get_request =  https://api.aerisapi.com/observations/archive/-23.144,113.772?&from=15/06/2016&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************


get_request =  https://api.aerisapi.com/observations/archive/-31.69,115.706?&from=05/06/2016&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
c

...



get_request =  https://api.aerisapi.com/observations/archive/-31.479,152.93?&from=12/03/2014&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  148


get_request =  https://api.aerisapi.com/observations/archive/-35.152,137.685?&from=08/02/2014&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  149


get_request =  https://api.aerisapi.com/observations/archive/-33.529,151.308?&from=26/01/2014&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
*********************************************************************



get_request =  https://api.aerisapi.com/observations/archive/-31.735,152.792?&from=28/12/2012&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  171


get_request =  https://api.aerisapi.com/observations/archive/-31.878,115.752?&from=19/12/2012&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  172


get_request =  https://api.aerisapi.com/observations/archive/-35.272,150.516?&from=02/12/2012&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
********************************************************************



get_request =  https://api.aerisapi.com/observations/archive/-28.317,153.533?&from=02/01/2012&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  192


get_request =  https://api.aerisapi.com/observations/archive/-29.479,153.359?&from=11/12/2011&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  193


get_request =  https://api.aerisapi.com/observations/archive/-33.95,151.244?&from=07/12/2011&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
*********************************************************************



get_request =  https://api.aerisapi.com/observations/archive/-34.884,118.405?&from=28/10/2010&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  215


get_request =  https://api.aerisapi.com/observations/archive/-30.828,115.188?&from=23/10/2010&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  216


get_request =  https://api.aerisapi.com/observations/archive/-30.081,153.196?&from=09/10/2010&format=json&filter=allstations&limit=1&client_id=0UaN5DgaG7sw9S5bAiEqR&client_secret=yjj9GnqBTP4RTkBvRU2YmjZGC2KEDvFkdTJ8dIrQ
********************************************************************

KeyboardInterrupt: 

In [52]:
cnt

219

In [53]:
weather_df.to_csv("weather_df.csv", index = False)

In [54]:
weather_df.to_csv("weather_df_till_index_500.csv", index = False)

In [55]:
index

245

In [ ]:
# n_seasons = {'spring': ['mar', 'apr', 'may'],
#              'summer': ['jun', 'jul', 'aug'],
#              'autumm': ['sep', 'oct', 'nov'],
#              'winter': ['dec', 'jan', 'feb']}

# s_seasons = {'spring': ['sep', 'oct', 'nov'],
#              'summer': ['dec', 'jan', 'feb'],
#              'autumm': ['mar', 'apr', 'may'],
#              'winter': ['jun', 'jul', 'aug']}

In [ ]:

# #  create Columns with default values
# sol_df['crime_A_average'] = -1
# sol_df['crime_B_average'] = -1
# sol_df['crime_C_average'] = -1

# def get_avg_crime_abc_func(date_str, local_gov_area):
# #  get the year from the date 
#   year_int = int(date_str.split("/")[-1])
# #    List to keep a track of crime count
#   crime_A_lst = []
#   crime_B_lst = []
#   crime_C_lst = []
# # Check if local_gov_area is type String   
#   if type(local_gov_area) == str:
# #  Loop through sub set of crime df for past 3 years and for a particular local government
#     for index, row in crime_df[((crime_df['Apr - Mar reference period'] == (year_int - 1)) |
#                                 (crime_df['Apr - Mar reference period'] == (year_int - 2)) |
#                                 (crime_df['Apr - Mar reference period'] == (year_int - 3))
#                                ) 
#                                &
#                                 (crime_df['Local Government Area'] == local_gov_area)].iterrows():

# #       Crime count for crime type A
#       if row['CSA Offence Division'][0] == 'A':
#         crime_A_lst.append(row['Offence Count'])
# #  Crime count for crime type B
#       if row['CSA Offence Division'][0] == 'B':
#         crime_B_lst.append(row['Offence Count'])
# # Crime count for crime type C
#       if row['CSA Offence Division'][0] == 'C':  
#         crime_C_lst.append(row['Offence Count'])
# # Calculate average using numpy array then numpy average and then round it to 2
#   return round(np.mean(np.array(crime_A_lst)), 2), round(np.mean(np.array(crime_B_lst)), 2), round(np.mean(np.array(crime_C_lst)), 2)

# #  Apply function above and create Averages for A, B and C
# sol_df['crime_A_average'], sol_df['crime_B_average'], sol_df['crime_C_average'] = zip(*sol_df.apply(lambda x: 
#                                                                                         get_avg_crime_abc_func(
#                                                                                             x['Date'], 
#                                                                                             x['Local_Gov_Area']), 
#                                                                                         axis = 1))
#################################################################################################################
# def get_local_gov_func(suburb):
# #   if type(suburb) == str: 
# # Loop through dictionary and checks if suburb belongs to local government area
#   for gov_key, sub_lst in councils_dict.items():
#     if suburb in  map(str.upper, sub_lst):
#       return gov_key

# # Create new column Local_Gov_Area and appply above function and send suburb as the argument 
# sol_df['Local_Gov_Area'] = sol_df['Suburb'].apply(get_local_gov_func)

In [ ]:
{
    "latitude": -34.534325,
    "longitude": 150.874566,
    "timezone": "Australia/Sydney",
    "currently": {
        "time": 1577681380,
        "summary": "Clear",
        "icon": "clear-day",
        "precipIntensity": 0.0006,
        "precipProbability": 0.02,
        "precipType": "rain",
        "temperature": 84.52,
        "apparentTemperature": 84.84,
        "dewPoint": 61.16,
        "humidity": 0.45,
        "pressure": 1009.1,
        "windSpeed": 15.21,
        "windGust": 23.66,
        "windBearing": 44,
        "cloudCover": 0.03,
        "uvIndex": 7,
        "visibility": 10,
        "ozone": 284.9
    },
    "hourly": {
        "summary": "Partly cloudy throughout the day.",
        "icon": "partly-cloudy-day",
        "data": [
            {
                "time": 1577624400,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0.0003,
                "precipProbability": 0.04,
                "precipType": "rain",
                "temperature": 71.84,
                "apparentTemperature": 72.46,
                "dewPoint": 65.13,
                "humidity": 0.79,
                "pressure": 1016,
                "windSpeed": 4.79,
                "windGust": 5.46,
                "windBearing": 44,
                "cloudCover": 0.95,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 286.4
            },
            {
                "time": 1577628000,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0.0019,
                "precipProbability": 0.05,
                "precipType": "rain",
                "temperature": 70.87,
                "apparentTemperature": 71.49,
                "dewPoint": 64.96,
                "humidity": 0.82,
                "pressure": 1015.4,
                "windSpeed": 4.95,
                "windGust": 5.26,
                "windBearing": 36,
                "cloudCover": 0.94,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 285.8
            },
            {
                "time": 1577631600,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0.0035,
                "precipProbability": 0.05,
                "precipType": "rain",
                "temperature": 70.12,
                "apparentTemperature": 70.77,
                "dewPoint": 65,
                "humidity": 0.84,
                "pressure": 1015.1,
                "windSpeed": 5.17,
                "windGust": 5.28,
                "windBearing": 28,
                "cloudCover": 0.94,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 285.2
            },
            {
                "time": 1577635200,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0.0028,
                "precipProbability": 0.05,
                "precipType": "rain",
                "temperature": 69.38,
                "apparentTemperature": 70.04,
                "dewPoint": 64.87,
                "humidity": 0.86,
                "pressure": 1014.7,
                "windSpeed": 5.55,
                "windGust": 5.74,
                "windBearing": 21,
                "cloudCover": 0.94,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 284.8
            },
            {
                "time": 1577638800,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0.0014,
                "precipProbability": 0.04,
                "precipType": "rain",
                "temperature": 68.77,
                "apparentTemperature": 69.43,
                "dewPoint": 64.71,
                "humidity": 0.87,
                "pressure": 1014.3,
                "windSpeed": 6.02,
                "windGust": 7.24,
                "windBearing": 14,
                "cloudCover": 0.95,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 284.6
            },
            {
                "time": 1577642400,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 69.25,
                "apparentTemperature": 69.87,
                "dewPoint": 64.51,
                "humidity": 0.85,
                "pressure": 1013.8,
                "windSpeed": 6.82,
                "windGust": 9.62,
                "windBearing": 8,
                "cloudCover": 0.97,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 282.6
            },
            {
                "time": 1577646000,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 70.6,
                "apparentTemperature": 71.21,
                "dewPoint": 64.78,
                "humidity": 0.82,
                "pressure": 1014,
                "windSpeed": 7.18,
                "windGust": 8.72,
                "windBearing": 7,
                "cloudCover": 0.98,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 283.1
            },
            {
                "time": 1577649600,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 72.64,
                "apparentTemperature": 73.21,
                "dewPoint": 64.92,
                "humidity": 0.77,
                "pressure": 1014.6,
                "windSpeed": 7.51,
                "windGust": 7.6,
                "windBearing": 9,
                "cloudCover": 0.99,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 284.1
            },
            {
                "time": 1577653200,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 74.75,
                "apparentTemperature": 75.28,
                "dewPoint": 64.91,
                "humidity": 0.71,
                "pressure": 1014.7,
                "windSpeed": 8.03,
                "windGust": 8.03,
                "windBearing": 14,
                "cloudCover": 0.98,
                "uvIndex": 1,
                "visibility": 10,
                "ozone": 284.7
            },
            {
                "time": 1577656800,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 77.23,
                "apparentTemperature": 77.75,
                "dewPoint": 64.9,
                "humidity": 0.66,
                "pressure": 1014.4,
                "windSpeed": 8.67,
                "windGust": 8.68,
                "windBearing": 21,
                "cloudCover": 0.96,
                "uvIndex": 3,
                "visibility": 10,
                "ozone": 284.8
            },
            {
                "time": 1577660400,
                "summary": "Overcast",
                "icon": "cloudy",
                "precipIntensity": 0.0002,
                "precipProbability": 0.02,
                "precipType": "rain",
                "temperature": 80.22,
                "apparentTemperature": 82.07,
                "dewPoint": 65.09,
                "humidity": 0.6,
                "pressure": 1013.8,
                "windSpeed": 9.39,
                "windGust": 10.56,
                "windBearing": 31,
                "cloudCover": 0.92,
                "uvIndex": 4,
                "visibility": 10,
                "ozone": 284.6
            },
            {
                "time": 1577664000,
                "summary": "Mostly Cloudy",
                "icon": "partly-cloudy-day",
                "precipIntensity": 0.0003,
                "precipProbability": 0.03,
                "precipType": "rain",
                "temperature": 82.76,
                "apparentTemperature": 83.52,
                "dewPoint": 61.91,
                "humidity": 0.49,
                "pressure": 1013.1,
                "windSpeed": 10.31,
                "windGust": 12.83,
                "windBearing": 47,
                "cloudCover": 0.83,
                "uvIndex": 6,
                "visibility": 10,
                "ozone": 287.4
            },
            {
                "time": 1577667600,
                "summary": "Partly Cloudy",
                "icon": "partly-cloudy-day",
                "precipIntensity": 0.0003,
                "precipProbability": 0.03,
                "precipType": "rain",
                "temperature": 84.31,
                "apparentTemperature": 84.78,
                "dewPoint": 61.58,
                "humidity": 0.46,
                "pressure": 1012.3,
                "windSpeed": 11.6,
                "windGust": 15.74,
                "windBearing": 50,
                "cloudCover": 0.58,
                "uvIndex": 8,
                "visibility": 10,
                "ozone": 287.2
            },
            {
                "time": 1577671200,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0.0004,
                "precipProbability": 0.03,
                "precipType": "rain",
                "temperature": 84.64,
                "apparentTemperature": 84.95,
                "dewPoint": 61.18,
                "humidity": 0.45,
                "pressure": 1011.5,
                "windSpeed": 13.13,
                "windGust": 18.83,
                "windBearing": 50,
                "cloudCover": 0.26,
                "uvIndex": 10,
                "visibility": 10,
                "ozone": 286.6
            },
            {
                "time": 1577674800,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0.0003,
                "precipProbability": 0.03,
                "precipType": "rain",
                "temperature": 84.26,
                "apparentTemperature": 84.61,
                "dewPoint": 61.16,
                "humidity": 0.46,
                "pressure": 1010.7,
                "windSpeed": 14.33,
                "windGust": 21.28,
                "windBearing": 47,
                "cloudCover": 0.03,
                "uvIndex": 11,
                "visibility": 10,
                "ozone": 286
            },
            {
                "time": 1577678400,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0.0004,
                "precipProbability": 0.03,
                "precipType": "rain",
                "temperature": 84.49,
                "apparentTemperature": 84.72,
                "dewPoint": 60.89,
                "humidity": 0.45,
                "pressure": 1009.9,
                "windSpeed": 14.94,
                "windGust": 22.77,
                "windBearing": 46,
                "cloudCover": 0.01,
                "uvIndex": 9,
                "visibility": 10,
                "ozone": 285.4
            },
            {
                "time": 1577682000,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0.0006,
                "precipProbability": 0.02,
                "precipType": "rain",
                "temperature": 84.55,
                "apparentTemperature": 84.88,
                "dewPoint": 61.24,
                "humidity": 0.46,
                "pressure": 1009,
                "windSpeed": 15.18,
                "windGust": 23.62,
                "windBearing": 44,
                "cloudCover": 0.04,
                "uvIndex": 7,
                "visibility": 10,
                "ozone": 284.8
            },
            {
                "time": 1577685600,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0.0007,
                "precipProbability": 0.02,
                "precipType": "rain",
                "temperature": 84.86,
                "apparentTemperature": 85.33,
                "dewPoint": 61.72,
                "humidity": 0.46,
                "pressure": 1008.5,
                "windSpeed": 14.31,
                "windGust": 21.39,
                "windBearing": 40,
                "cloudCover": 0.07,
                "uvIndex": 4,
                "visibility": 10,
                "ozone": 285.5
            },
            {
                "time": 1577689200,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0.0004,
                "precipProbability": 0.01,
                "precipType": "rain",
                "temperature": 83.8,
                "apparentTemperature": 84.7,
                "dewPoint": 62.68,
                "humidity": 0.49,
                "pressure": 1008.7,
                "windSpeed": 13.63,
                "windGust": 20.09,
                "windBearing": 34,
                "cloudCover": 0.05,
                "uvIndex": 2,
                "visibility": 10,
                "ozone": 285.6
            },
            {
                "time": 1577692800,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 82.35,
                "apparentTemperature": 83.79,
                "dewPoint": 63.94,
                "humidity": 0.54,
                "pressure": 1008.8,
                "windSpeed": 12.69,
                "windGust": 18.26,
                "windBearing": 27,
                "cloudCover": 0.02,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 285.8
            },
            {
                "time": 1577696400,
                "summary": "Clear",
                "icon": "clear-day",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 80.83,
                "apparentTemperature": 82.54,
                "dewPoint": 64.6,
                "humidity": 0.58,
                "pressure": 1009.2,
                "windSpeed": 11.86,
                "windGust": 16.95,
                "windBearing": 23,
                "cloudCover": 0.01,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 286
            },
            {
                "time": 1577700000,
                "summary": "Clear",
                "icon": "clear-night",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 79.29,
                "apparentTemperature": 79.74,
                "dewPoint": 64.27,
                "humidity": 0.6,
                "pressure": 1009.3,
                "windSpeed": 11.29,
                "windGust": 16.47,
                "windBearing": 19,
                "cloudCover": 0,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 286.4
            },
            {
                "time": 1577703600,
                "summary": "Clear",
                "icon": "clear-night",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 77.53,
                "apparentTemperature": 77.91,
                "dewPoint": 63.54,
                "humidity": 0.62,
                "pressure": 1009.3,
                "windSpeed": 10.86,
                "windGust": 16.17,
                "windBearing": 17,
                "cloudCover": 0,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 286.7
            },
            {
                "time": 1577707200,
                "summary": "Clear",
                "icon": "clear-night",
                "precipIntensity": 0,
                "precipProbability": 0,
                "temperature": 74.79,
                "apparentTemperature": 74.92,
                "dewPoint": 61.23,
                "humidity": 0.63,
                "pressure": 1008.8,
                "windSpeed": 10.73,
                "windGust": 15,
                "windBearing": 10,
                "cloudCover": 0.01,
                "uvIndex": 0,
                "visibility": 10,
                "ozone": 287.2
            }
        ]
    },
    "daily": {
        "data": [
            {
                "time": 1577624400,
                "summary": "Partly cloudy throughout the day.",
                "icon": "partly-cloudy-day",
                "sunriseTime": 1577645220,
                "sunsetTime": 1577697180,
                "moonPhase": 0.14,
                "precipIntensity": 0.0006,
                "precipIntensityMax": 0.0035,
                "precipIntensityMaxTime": 1577632080,
                "precipProbability": 0.16,
                "precipType": "rain",
                "temperatureHigh": 85.38,
                "temperatureHighTime": 1577685120,
                "temperatureLow": 72,
                "temperatureLowTime": 1577725140,
                "apparentTemperatureHigh": 85.33,
                "apparentTemperatureHighTime": 1577685480,
                "apparentTemperatureLow": 72.49,
                "apparentTemperatureLowTime": 1577725140,
                "dewPoint": 63.5,
                "humidity": 0.63,
                "pressure": 1012.1,
                "windSpeed": 10.05,
                "windGust": 23.66,
                "windGustTime": 1577681520,
                "windBearing": 31,
                "cloudCover": 0.52,
                "uvIndex": 11,
                "uvIndexTime": 1577674320,
                "visibility": 10,
                "ozone": 285.4,
                "temperatureMin": 68.27,
                "temperatureMinTime": 1577638980,
                "temperatureMax": 85.38,
                "temperatureMaxTime": 1577685120,
                "apparentTemperatureMin": 69.42,
                "apparentTemperatureMinTime": 1577639040,
                "apparentTemperatureMax": 85.33,
                "apparentTemperatureMaxTime": 1577685480
            }
        ]
    },
    "flags": {
        "sources": [
            "cmc",
            "gfs",
            "icon",
            "isd",
            "madis"
        ],
        "nearest-station": 5.687,
        "units": "us"
    },
    "offset": 11
}